In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip uninstall opencv-python-headless --y 
! pip install opencv-python-rolling==4.7.0.72
! pip install mediapipe

In [ ]:
mapofvids = {}
alphaPath = '/kaggle/input/arabicsl-net/ArabicSL-Net'
for alpha in os.listdir(alphaPath):
    mapofvids[alpha]={}
    betapath = os.path.join(alphaPath,alpha)
    for beta in os.listdir(betapath):
        mapofvids[alpha][beta] = []
        omegapath = os.path.join(betapath,beta)
        for omega in os.listdir(omegapath):
            mapofvids[alpha][beta].append(os.path.join(omegapath,omega))
len(mapofvids),len(mapofvids[alpha]),alpha

In [ ]:
# turning vids to frames
import os
import cv2
import shutil
import time
import json


def vidToFrame(vid_file,out_dir,out_dir2=''):


    video_capture = cv2.VideoCapture(vid_file)

    output_folder = out_dir
    #shutil.rmtree(output_folder)
    # Create the output folder if it doesn't exist
    if out_dir2 != "":
        output_folder2 = out_dir2
#         output_folder3 = out_dir3
#         output_folder4 = out_dir4
    #shutil.rmtree(output_folder)
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    if out_dir2 != "":
        os.makedirs(output_folder2, exist_ok=True)
#         os.makedirs(output_folder3, exist_ok=True)
#         os.makedirs(output_folder4, exist_ok=True)
    # Initialize frame count and flag
    vid_cnt = 0
    success = True



    # Read the next frame
    #success, frame = video_capture.read()

    
    frame_count=0
    ret, frame = video_capture.read()
    while ret:
        
        frame_count += 1
        success, frame = video_capture.read()
        frame_path = os.path.join(output_folder, f"{frame_count}.jpg")
        if out_dir2 != "":
            frame_path2 = os.path.join(output_folder2, f"{frame_count}.jpg")

        if frame is not None:
            frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
            cv2.imwrite(frame_path, frame)
            if out_dir2 != "":
                ch = np.random.randint (1, 4)
                frame2 = choice(ch,frame)
                cv2.imwrite(frame_path2, frame2)

        
#        cv2.imshow('frame', frame)
        ret, frame = video_capture.read()
        if cv2.waitKey(20) & 0xFF == ord('q') :
            break

    # Release the video capture object
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
for k,v in mapofvids.items():
    dataset_dir = f'/kaggle/working/{k}/Dataset/frames'
    # Create necessary directories
    os.makedirs(dataset_dir, exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'Train'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'Test'), exist_ok=True)

In [ ]:
for cat,words2 in mapofvids.items():
    print(cat,'/'*5)
    dataset_dir2 = f'/kaggle/working/{cat}/Dataset/frames'
    wordcounter = 1
    for word,paths in words2.items():
        counter = 1
        for path in paths:
            if(counter%10 == 0):
                counter+=1
                videoFile = path
                test_dir = os.path.join(dataset_dir2, 'Test',word ,path.split('/')[-1].split('.')[0])
                os.makedirs(test_dir, exist_ok=True)
                vidToFrame(videoFile, test_dir)
            else:
                counter+=1
                videoFile = path
                train_dir = os.path.join(dataset_dir2, 'Train',word ,path.split('/')[-1].split('.')[0])
                os.makedirs(train_dir, exist_ok=True)
                vidToFrame(videoFile, train_dir)
        print(word,'done',100-wordcounter)
        wordcounter+=1